In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import statsmodels.api as sm
import hockey_scraper
import pickle
import time
import random
import datetime
yesterday = datetime.date.today()- datetime.timedelta(days=1)
pd.set_option('display.max_columns', None)

In [ ]:
def master_function():
    # bring in goalie game logs 
    infile = open("data/goalie_game_log_df.pickle",'rb')
    goalie_game_log = pickle.load(infile)
    infile.close()
    
    #get goalie stats from prior days games
    get_goalie_data(get_goalie_ids())
    

In [50]:
#game boxscore url
url = 'https://statsapi.web.nhl.com/api/v1/game/{}/boxscore'.format(game_id)

In [ ]:
#roster url for future uses 
'https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster&season=20142015'

In [2]:
#schedule url, date format: 2018-01-09

def get_yesterdays_game_ids():
    yesterday = datetime.date.today()- datetime.timedelta(days=1)
    url = 'https://statsapi.web.nhl.com/api/v1/schedule?date={}'.format(yesterday.strftime('%Y-%m-%d'))
    r = requests.get(url)
    y_sched = r.json()

    game_id = []
    for game in y_sched['dates'][0]['games']:
        game_id.append(game['gamePk'])
    
    return game_id

In [3]:
get_yesterdays_game_ids()

[2020030323]

In [73]:
#get goalie ids from yesterdays games
def get_goalie_ids():
    goalie_id = {}
    for game_id in get_yesterdays_game_ids():
        url = 'https://statsapi.web.nhl.com/api/v1/game/{}/boxscore'.format(game_id)
        r = requests.get(url)
        box_score = r.json()
        for e in ['home', 'away']:
            temp_id = []
            temp_id.extend(box_score['teams'][e]['goalies'])
            for pid in temp_id:
                name = box_score['teams'][e]['players']['ID'+str(pid)]['person']['fullName']
                goalie_id[name] = pid
    return goalie_id
    

        
    
    

In [74]:
get_goalie_ids()

{'Semyon Varlamov': 8473575, 'Andrei Vasilevskiy': 8476883}

In [99]:
def get_goalie_data(goalie_ids, start_year =20212022, end_year = 20212022):
    counter = 0
    for name, gid in goalie_ids.items():

        sequence = [x/10 for x in range(60, 120)]
        time.sleep(random.choice(sequence))
        url = 'https://www.naturalstattrick.com/playerreport.php?fromseason={}&thruseason={}&playerid={}&sit=all&stype=2&stdoi=oi&rate=n&v=g'.format(start_year, end_year, gid)
        #due to number of http requests, NST may ban your IP before the loop finishes. I needed to use a VPN to get around this. If IP gets banned, this function will still return the current DF and you can call the function again and pass in an updated goalie dictionary to get the rest
        try:
            individual_df = pd.read_html(url)[0]
            individual_df['Name'] = name
            individual_df['ID'] = gid

        except:
            print(f'Ended before {name}')
            return all_goalies

        if counter == 0:
            all_goalies = individual_df

        elif counter != 0:
            all_goalies = pd.concat([all_goalies, individual_df])



        counter +=1
    
    return all_goalies

In [76]:
def goalie_features(df, rolling_games = 40):
    rolling_games = rolling_games
    min_games = 10
    df['Date'] = df['Game'].apply(lambda x: pd.to_datetime(x[0:10]))
    df['Team_Key'] = df['Team'].astype(str)+'_'+df['Date'].astype(str)
    
    df['Rolling_TOI'] = df.groupby('ID')['TOI'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_FA'] = df.groupby('ID')['FA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_SA'] = df.groupby('ID')['SA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_GA'] = df.groupby('ID')['GA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_xGA'] = df.groupby('ID')['xGA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_HDCA'] = df.groupby('ID')['HDCA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_HDGA'] = df.groupby('ID')['HDGA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    
    df['Goalie_FenwickSV%'] =  (df['Rolling_FA'] - df['Rolling_GA']) /  df['Rolling_FA']
    df['Goalie_GSAx'] = df['Rolling_xGA'] - df['Rolling_GA']
    df['Goalie_GSAx/60'] =  df['Goalie_GSAx']*60 /  df['Rolling_TOI']
    df['Goalie_HDCSV%'] = (df['Rolling_HDCA'] - df['Rolling_HDGA'] ) / df['Rolling_HDCA'] 
    return df

In [79]:
infile = open("data/goalie_game_log_df.pickle",'rb')
goalie_game_log = pickle.load(infile)
infile.close()

In [ ]:
def update_goalie_logs():
    # bring in goalie game logs 
    infile = open("data/goalie_game_log_df.pickle",'rb')
    goalie_game_log = pickle.load(infile)
    infile.close()
    
    #get goalie stats for goalies that played yesterday
    yesterdays_goalies_df = get_goalie_data(get_goalie_ids())
    yesterdays_goalies_df['Date'] = yesterdays_goalies_df['Game'].apply(lambda x: pd.to_datetime(x[0:10]))
    
    #concact yesterdays goalies stats for just yesterday's games 
    goalie_game_log = pd.concat([goalie_game_log, yesterdays_goalies_df[yesterdays_goalies_df['Date'] == yesterday]])
    #recalculate stats for features
    goalie_game_log = goalie_features(goalie_game_log)
    
    return goalie_game_log